In [1]:
library(ggtree)
library(treeio)
library(phangorn)
library(data.table)
library(ips)
library(stringr)
library(Biostrings)
library(ggplot2)
library(ggpubr)


Registered S3 methods overwritten by 'treeio':
  method              from    
  MRCA.phylo          tidytree
  MRCA.treedata       tidytree
  Nnode.treedata      tidytree
  Ntip.treedata       tidytree
  ancestor.phylo      tidytree
  ancestor.treedata   tidytree
  child.phylo         tidytree
  child.treedata      tidytree
  full_join.phylo     tidytree
  full_join.treedata  tidytree
  groupClade.phylo    tidytree
  groupClade.treedata tidytree
  groupOTU.phylo      tidytree
  groupOTU.treedata   tidytree
  is.rooted.treedata  tidytree
  nodeid.phylo        tidytree
  nodeid.treedata     tidytree
  nodelab.phylo       tidytree
  nodelab.treedata    tidytree
  offspring.phylo     tidytree
  offspring.treedata  tidytree
  parent.phylo        tidytree
  parent.treedata     tidytree
  root.treedata       tidytree
  rootnode.phylo      tidytree
  sibling.phylo       tidytree

ggtree v3.6.2 For help: https://yulab-smu.top/treedata-book/

If you use the ggtree package suite in published rese

In [16]:
detectCores()

[1] 160

In [2]:
##------------------------------------------------------------------------------
##------------------------------------------------------------------------------
# """
# Calculate mutation table
# input: binary table from phy files
# output: mutation count at each position as a data frame
# """
.mutTable <- function(binaryTable){
  binaryTable = data.frame(binaryTable,do.call(rbind,str_split(binaryTable$V2,"")))
  binaryTable <- binaryTable[,-2]
  setDF(binaryTable)
  colnames(binaryTable)[1] <- "name"
  s = c()
  for(i in 2:ncol(binaryTable)) {
    binaryTable[,i] <- as.numeric(binaryTable[,i])
    s <- c(s, sum(binaryTable[,i]))
  }
  s <- which(s>0)+1
  binaryTable <- binaryTable[, c(1,s)]
  new <- lapply(as.list(unique(binaryTable$name)),function(my.name){
    df <- data.frame()
    temp <- binaryTable[binaryTable$name == my.name,]
    temp <- temp[,which(temp>0)]
    if(!is.null(ncol(temp))){
      for(j in 2:ncol(temp)){
        df <- rbind(df, data.table(name = my.name, 
                                   pos = as.numeric(gsub("X","",colnames(temp)[j])), 
                                   mut = 1))
      }
    }
    return(df)
  })
  df <- rbindlist(new)
  return(df)
}

##------------------------------------------------------------------------------
##------------------------------------------------------------------------------
.plotTree_rect_2panel <- function(samples, PHYDIR, TREEDIR,labNames,sampleSize=TRUE){
  setwd(TREEDIR)
  for(i in 1:length(samples)){
    message("Calculating ",samples[i], "...")
    biTable <- read.table(paste0(PHYDIR,samples[i],"_filtered_re_rescale.substituted_tumor_treev2.3.0.nwk.modified.phy"),header = F,
                          colClasses=c("character", "character"),sep = " ",skip=1)
    if(!file.exists(paste0(TREEDIR,samples[i],"_genotype_info_ML.txt"))){
      ##---generate info table
      tipnames = biTable$V1
      if(length(strsplit(tipnames[2],"_")[[1]]) ==1){
        d1 <- substr(tipnames, 1,1)
      }else{
        d1=sapply(tipnames,function(x){
          y = strsplit(x,"_")[[1]]
          if(length(y)>2){
            paste0(y[1],"_",y[2])
          }else{y[1]}
        },USE.NAMES = FALSE)
      }
      d1 <- data.frame(class = d1)
      d1$id <- tipnames
      d1 <- d1[,c(2,1)]
      write.table(d1, file = paste0(TREEDIR,samples[i],"_genotype_info_ML.txt"),sep= "\t", row.names = F, quote = F)
    }
    df_info <- read.table(paste0(TREEDIR,samples[i],"_genotype_info_ML.txt"), sep = "\t", header = T)
    table(df_info$class)
    grp <- sapply(as.list(unique(df_info$class)),function(i){df_info[df_info$class==i,]$id})
    names(grp) <- unique(df_info$class)
    
    # df_info$class <- factor(df_info$class, levels = c("T","r","N"))
    print(table(df_info$class))
    if(!file.exists(paste0(samples[i],"_mutation_table.txt"))){
      df <- .mutTable(biTable)
      write.table(df, file = paste0(samples[i],"_mutation_table.txt"),sep = "\t",
                  row.names = F, quote = F)
    }
    df <- read.delim(paste0(samples[i],"_mutation_table.txt"), header = T)
    setDT(df)
    df2 <- lapply(as.list(as.list(unique(df$name))),function(nC){
      data.table(name=nC, numMut = df[name==nC, sum(mut)])
    })
    df2 <- rbindlist(df2)
    tree <- read.tree(paste0(samples[i],"_filtered_re_rescale.substituted_tumor_treev2.3.0.nwk"))
    ##------------------------------
    ## trim tree
    # re <- names(tip[which(tip <= quantile(tip, 0.01) | tip >= quantile(tip, 0.99))])
    tip <- tipHeights(tree)
    re <- names(tip[which(tip >100)])
    tree <- drop.tip(tree,re)
    tree <- drop.tip(tree,"ref")
    s = tree$tip.label
    s <- substr(s, 1,1)
    df <- df[!name %in% re]
    if(length(strsplit(df$name[2],"_")[[1]]) ==1){
      df$group <- substr(df$name, 1,1)
      table(df$group)
    }else{
      df$group=sapply(df$name,function(x){
        y = strsplit(x,"_")[[1]]
        if(length(y)>2){
          paste0(y[1],"_",y[2])
        }else{y[1]}
      },USE.NAMES = FALSE)
      table(df$group)
    }
    mycols1 = c(N="#4F9EC2",T="#C44754")#N="#2885A9",T="#9F1F31" 
    mycols2 = c(N="#4F9EC2",P="#C44754")#ref="#b3de69", 
    mycols3 = c("#C44754","#66A61E","#1B9E77","#7570B3","#D95F02","#E7298A","#E6AB02","#A6761D","#fb9a99","#666666")##Dark2 panel
    ##--------------------------------------------------------------------------
    df_info <- df_info[df_info$class != "ref",]
    if(length(unique(df_info$class))==2 & "N" %in% unique(df_info$class) & 
       "T" %in% unique(df_info$class)){
      mycols = mycols1
    }else if(length(unique(df_info$class))==2 & "N" %in% unique(df_info$class) & 
             "P" %in% unique(df_info$class)){
      mycols = mycols2
    }else{
      if("N" %in% unique(df_info$class)){mycols = c(N="#4F9EC2")}else{mycols = c()}
      classes = unique(df_info$class)
      classes <- classes[!classes %in% c("N","ref")]
      for(my.class in 1:length(classes)){
        temp = mycols3[my.class]
        names(temp) <- classes[my.class]
        mycols <- c(mycols, temp)
      }
    }
    
    p <- ggtree(tree, layout = "rectangular")#+ theme_tree2()#+xlim(0,lims)
    p <- p %<+% df_info + geom_tippoint(aes(color=class))+
      scale_color_manual(values=mycols,name=NULL)
    p <- p+geom_facet(panel = "Mutation", data = df, geom = geom_point,
                      mapping=aes(x = pos, color = class), shape = ".")+
      # geom_facet(panel = "Mutation numbers", data = df2, geom = geom_bar, 
      #            mapping = aes(x=numMut, fill=class,name=NULL),orientation="y", stat="identity")+
      scale_fill_manual(values=mycols,name=NULL)+
      ggtitle(labNames[i])+theme_classic()+#theme_tree2() + 
      theme(text = element_text(size=26),plot.title = element_text(hjust = 0.5, size = 30))
    if(sampleSize == TRUE){
      p <- p+ggtitle(paste0(labNames[i]," (n=",scales::comma(nrow(df_info)),")"))
    }else{
      p <- p+ggtitle(labNames[i])
    }
    
    print(p)
  }
}

##------------------------------------------------------------------------------
##------------------------------------------------------------------------------
##------------------------------------------------------------------------------
##------------------------------------------------------------------------------
.plotTree <- function(samples, PHYDIR, TREEDIR,labNames, layOut = "rectagular", 
                      branchLength = "branch.length",colorTip=TRUE,logBranch=FALSE,sampleSize=TRUE){
  setwd(TREEDIR)
  p2 <- list()
  for(i in 1:length(samples)){
    message("Calculating ",samples[i], "...")
    if(!file.exists(paste0(TREEDIR,samples[i],"_genotype_info_ML.txt"))){
      biTable <- read.table(paste0(PHYDIR,samples[i],"_filtered_re_rescale.substituted_tumor_treev2.3.0.nwk.modified.phy"),header = F,
                            colClasses=c("character", "character"),sep = " ",skip=1)
      ##---generate info table
      tipnames = biTable$V1
      if(length(strsplit(tipnames[2],"_")[[1]]) ==1){
        d1 <- substr(tipnames, 1,1)
      }else{
        d1=sapply(tipnames,function(x){
          y = strsplit(x,"_")[[1]]
          if(length(y)>2){
            paste0(y[1],"_",y[2])
          }else{y[1]}
        },USE.NAMES = FALSE)
      }
      d1 <- data.frame(class = d1)
      d1$id <- tipnames
      d1 <- d1[,c(2,1)]
      write.table(d1, file = paste0(TREEDIR,samples[i],"_genotype_info_ML.txt"),sep= "\t", row.names = F, quote = F)
    }
    df_info <- read.table(paste0(samples[i],"_genotype_info_ML.txt"), sep = "\t", header = T)
    print(table(df_info$class))
    grp <- sapply(as.list(unique(df_info$class)),function(i){df_info[df_info$class==i,]$id})
    names(grp) <- unique(df_info$class)
    grp <- grp[names(grp) != "ref"]
    tree <- read.tree(paste0(samples[i],"_filtered_re_rescale.substituted_tumor_treev2.3.0.nwk"))
    ##------------------------------
    ## trim tree
    # re <- names(tip[which(tip <= quantile(tip, 0.01) | tip >= quantile(tip, 0.99))])
    tip <- tipHeights(tree)
    if(logBranch){
        tree$edge.length <- log(tree$edge.length+1)
    }
    re <- names(tip[which(tip >100)])
    tree <- drop.tip(tree,re)
    tree <- drop.tip(tree,"ref")
    s = tree$tip.label
    s <- substr(s, 1,1)
    
    mycols1 = c(N="#2885A9",T="#9F1F31")#ref="#b3de69", 
    mycols2 = c(N="#2885A9",P="#9F1F31")#ref="#b3de69", 
    mycols3 = c("#9F1F31","#66A61E","#1B9E77","#7570B3","#D95F02","#E7298A","#E6AB02","#A6761D","#fb9a99","#666666")##fb9a99#Dark2 panel
    #mycols3 = c("#2885A9")##fb9a99#Dark2 panel
    ##--------------------------------------------------------------------------
    df_info <- df_info[df_info$class != "ref",]
    if(length(unique(df_info$class))==2 & "N" %in% unique(df_info$class) & 
       "T" %in% unique(df_info$class)){
      mycols = mycols1
    }else if(length(unique(df_info$class))==2 & "N" %in% unique(df_info$class) & 
             "P" %in% unique(df_info$class)){
      mycols = mycols2
    }else{
      if("N" %in% unique(df_info$class)){mycols = c(N="#2885A9")}else{mycols = c()}
      classes = unique(df_info$class)
      classes <- classes[!classes %in% c("N","ref")]
      for(my.class in 1:length(classes)){
        temp = mycols3[my.class]
        names(temp) <- classes[my.class]
        mycols <- c(mycols, temp)
      }
    }
    ##--------------------------------------------------------------------------
    if(colorTip){
      rownames(df_info) <- df_info$id
      # p <- ggtree(tree, layout = "circular", branch.length = branchLength)
      p <- ggtree(tree,aes(color=class), layout=layOut,branch.length = branchLength,size=0.1) %<+% df_info  
      p <- p+scale_color_manual(values=mycols,name=NULL,na.value = "#A2A2A2")
    }else{
      p <- ggtree(tree, layout = layOut, branch.length = branchLength)
      p <- p+scale_color_manual(values=mycols,name=NULL)
    }

    p <- gheatmap(p, df_info[,"class",drop=FALSE], offset=.8, width=.2, colnames = FALSE, color=NA) +
    theme(legend.position ="right",legend.key.size = unit(0.5, 'cm'))+
    scale_fill_manual(values=mycols,name=NULL)
    p <- p+theme(text = element_text(size=12),plot.title = element_text(hjust = 0.5, size = 14))
    ##--------------------------------------------------------------------------
        
    ##--------------------------------------------------------------------------
    if(sampleSize == TRUE){
      p <- p+ggtitle(paste0(labNames[i]," (n=",scales::comma(nrow(df_info)),")"))
    }else{
      p <- p+ggtitle(labNames[i])
    }
    p2[[i]] <- p
  }
  return(p2)
}


## 1. filteredNT_re

In [12]:
##--------------------------------------------------------------------------------------------------
samples=c("151_T4","47_T6","47_T8","17_T4","19_T4","18_T4","19_T5","50_T","18_T3","142_T2",
          "47_T4","19_T1","132_T1","47_T5","148_T3","65_T3","47_T1",#"142_T5-4",
          "148_T1","5_T","19_T3","66_T","4_T","16_T","142_T1","49_T1","49_T3","65_T1")
labNames=c("151_T4","47_T6","47_T8","17_T4","19_T4","18_T4","19_T5","50_T","18_T3","142_T2",
          "47_T4","19_T1","132_T1","47_T5","148_T3","65_T3","47_T1",#"142_T5-4",
          "148_T1","5_T","19_T3","66_T","4_T","16_T","142_T1","49_T1","49_T3","65_T1")
PHYDIR = "/data2/kantian/LineageTracing/SMALT/0.Results/5.trimmed_trees/"
TREEDIR = PHYDIR
OUTDIR="/data2/kantian/LineageTracing/SMALT/0.Results/figures/"
##--------------------------------------------------------------------------------------------------
pdf(file = paste0(OUTDIR,"5.1.plotTree_NTfiltered_re_rect_2panels_trimmed.pdf"), height = 12, width = 12)
.plotTree_rect_2panel(samples, PHYDIR,TREEDIR,labNames)
dev.off()
p3 <- .plotTree(samples, PHYDIR,TREEDIR,labNames,layOut = "circular", branchLength = "branch.length",colorTip=TRUE,logBranch=TRUE,sampleSize = TRUE)
pdf(file = paste0(OUTDIR,"5.1.plotTree_NTfiltered_re_circular_trimmed.pdf"), height = 10, width = 15)
ggarrange(p3[[1]],p3[[2]],p3[[3]],p3[[4]],p3[[5]],p3[[6]],p3[[7]],p3[[8]],p3[[9]],p3[[10]],
          p3[[11]],p3[[12]],p3[[13]],p3[[14]],p3[[15]],p3[[16]],p3[[17]],p3[[18]],p3[[19]],p3[[20]],
          p3[[21]],p3[[22]],p3[[23]],p3[[24]],p3[[25]],p3[[26]],p3[[27]],p3[[28]],
            nrow = 3, ncol=4)
dev.off()


Calculating 151_T4...




  N ref  T4 
288   1 406 


ℹ invalid tbl_tree object. Missing column: parent,node.

ℹ invalid tbl_tree object. Missing column: parent,node.

Calculating 47_T6...




   N  ref   T6 
1043    1  101 


ℹ invalid tbl_tree object. Missing column: parent,node.

ℹ invalid tbl_tree object. Missing column: parent,node.

Calculating 47_T8...



...


   N  ref   T3 
 141    1 1568 


! # Invaild edge matrix for <phylo>. A <tbl_df> is returned.

! # Invaild edge matrix for <phylo>. A <tbl_df> is returned.

ℹ invalid tbl_tree object. Missing column: label.

ℹ invalid tbl_tree object. Missing column: label.

! # Invaild edge matrix for <phylo>. A <tbl_df> is returned.

! # Invaild edge matrix for <phylo>. A <tbl_df> is returned.

Scale for y is already present.
Adding another scale for y, which will replace the existing scale.
Scale for fill is already present.
Adding another scale for fill, which will replace the existing scale.
Calculating 65_T1...




  N ref  T1 
251   1 115 


! # Invaild edge matrix for <phylo>. A <tbl_df> is returned.

! # Invaild edge matrix for <phylo>. A <tbl_df> is returned.

ℹ invalid tbl_tree object. Missing column: label.

ℹ invalid tbl_tree object. Missing column: label.

! # Invaild edge matrix for <phylo>. A <tbl_df> is returned.

! # Invaild edge matrix for <phylo>. A <tbl_df> is returned.

Scale for y is already present.
Adding another scale for y, which will replace the existing scale.
Scale for fill is already present.
Adding another scale for fill, which will replace the existing scale.


ERROR: Error in p3[[28]]: subscript out of bounds


In [13]:
pdf(file = paste0(OUTDIR,"5.1.plotTree_NTfiltered_re_circular_trimmed.pdf"), height = 10, width = 15)
ggarrange(p3[[1]],p3[[2]],p3[[3]],p3[[4]],p3[[5]],p3[[6]],p3[[7]],p3[[8]],p3[[9]],p3[[10]],
          p3[[11]],p3[[12]],p3[[13]],p3[[14]],p3[[15]],p3[[16]],p3[[17]],p3[[18]],p3[[19]],p3[[20]],
          p3[[21]],p3[[22]],p3[[23]],p3[[24]],p3[[25]],p3[[26]],p3[[27]],
            nrow = 3, ncol=4)
dev.off()


$`1`

$`2`

$`3`

attr(,"class")
[1] "list"      "ggarrange"

png 
  2